<a href="https://colab.research.google.com/github/HayunJin/open_api/blob/main/%EC%9D%BC%EB%B3%84%EC%BD%94%EB%A1%9C%EB%82%98%ED%99%95%EC%A7%84%EC%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import unquote, quote_plus, urlencode

In [2]:
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
servicekey = '서비스키''
decodekey = unquote(servicekey)

In [3]:
startCreateDt = 20210105
endCreateDt = 20210405

query_params = '?' + urlencode({
    quote_plus('serviceKey'): decodekey,
    quote_plus('startCreateDt'): startCreateDt,
    quote_plus('endCreateDt'): endCreateDt
})

URL = url + query_params

In [4]:
request = Request(URL)
page = urlopen(request).read()

In [5]:
soup = BeautifulSoup(page, 'html.parser')
parse = soup.find_all(['decidecnt', 'statedt'])  # decidecnt 태그와 statedt 태그 다 찾기

df = []
for dt, cnt in zip(parse[1::2], parse[0::2]):
    df.append([dt.text, int(cnt.text)])  # 태그 안의 값은 ".text"를 통해 접근할 수 있음

df = pd.DataFrame(df, columns=['날짜', '누적확진자'])
df = df.sort_values(by='날짜').reset_index(drop=True) 
df['일별확진자'] = df['누적확진자'].diff()
df = df.loc[1:].reset_index(drop=True)  # 맨 앞의 데이터는 지우기 (일별 확진자가 계산되지 않았기 때문에)
df['일별확진자'] = df['일별확진자'].astype(int)


,날짜,누적확진자,일별확진자
0,20210106,65816,838
1,20210107,66684,868
2,20210108,67358,674
3,20210109,67999,641
4,20210110,68663,664
...,...,...,...
85,20210401,103636,551
86,20210402,104193,557
87,20210403,104736,543
88,20210404,105279,543


In [ ]:
df

In [16]:
df.to_csv('corona_210405.csv', sep=',')   # missing data representation (결측값 표기)

In [12]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive
